# Getting Started with Pulsim

This notebook introduces the basics of circuit simulation with Pulsim.

## Contents
1. Creating a Simple RC Circuit
2. Running a Transient Simulation
3. Analyzing and Plotting Results
4. Working with JSON Netlists

## 1. Creating a Simple RC Circuit

Let's start by creating a simple RC low-pass filter circuit.

In [1]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

# Create a new circuit
circuit = pulsim.Circuit("RC Low-Pass Filter")

# Add components
# Voltage source: 1V DC at node 'in' referenced to ground '0'
circuit.add_voltage_source("Vin", "in", "0", 1.0)

# Resistor: 1kΩ between 'in' and 'out'
circuit.add_resistor("R1", "in", "out", 1000)  # 1kΩ

# Capacitor: 1µF between 'out' and ground
circuit.add_capacitor("C1", "out", "0", 1e-6)  # 1µF

print(f"Circuit: {circuit.name}")
print(f"Components: {circuit.component_count}")
print(f"Nodes: {circuit.nodes}")

ModuleNotFoundError: No module named 'pulsim'

## 2. Running a Transient Simulation

Now let's simulate the circuit response to a step input.

In [ ]:
# Configure simulation options
options = pulsim.SimulationOptions(
    stop_time=5e-3,      # 5ms simulation
    timestep=1e-6,       # 1µs timestep
    abstol=1e-12,        # Absolute tolerance
    reltol=1e-3          # Relative tolerance
)

# Run simulation
result = pulsim.simulate(circuit, options)

print(f"Simulation completed!")
print(f"Total timesteps: {result.total_steps}")
print(f"Newton iterations: {result.newton_iterations_total}")

## 3. Analyzing and Plotting Results

In [ ]:
# Extract data
time = result.time * 1e3  # Convert to ms
v_in = result.voltages["in"]
v_out = result.voltages["out"]

# Calculate time constant
R = 1000  # 1kΩ
C = 1e-6  # 1µF
tau = R * C  # Time constant = 1ms

# Theoretical response
v_theory = 1.0 * (1 - np.exp(-result.time / tau))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(time, v_in, 'b-', label='V_in', linewidth=2)
plt.plot(time, v_out, 'r-', label='V_out (simulated)', linewidth=2)
plt.plot(time, v_theory, 'g--', label='V_out (theoretical)', linewidth=2)

plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('RC Low-Pass Filter Step Response')
plt.legend()
plt.grid(True)
plt.xlim([0, 5])
plt.ylim([0, 1.1])

# Add time constant annotation
plt.axhline(y=0.632, color='gray', linestyle=':', alpha=0.7)
plt.axvline(x=tau*1e3, color='gray', linestyle=':', alpha=0.7)
plt.annotate(f'τ = {tau*1e3:.1f} ms', xy=(tau*1e3, 0.632), 
             xytext=(tau*1e3 + 0.5, 0.5), fontsize=10,
             arrowprops=dict(arrowstyle='->', color='gray'))

plt.tight_layout()
plt.show()

In [ ]:
# Verify accuracy
max_error = np.max(np.abs(v_out - v_theory))
print(f"Maximum error vs theoretical: {max_error:.6f} V")
print(f"Relative error: {max_error/1.0 * 100:.4f}%")

## 4. Working with JSON Netlists

Pulsim can also read circuits from JSON files.

In [ ]:
# Define a circuit as JSON
netlist_json = '''
{
  "name": "RC Filter with Pulse Input",
  "components": [
    {"type": "V", "name": "Vin", "nodes": ["in", "0"],
     "waveform": {"type": "pulse", "v1": 0, "v2": 1, "period": 2e-3, "pw": 1e-3}},
    {"type": "R", "name": "R1", "nodes": ["in", "out"], "value": 1000},
    {"type": "C", "name": "C1", "nodes": ["out", "0"], "value": 1e-6}
  ],
  "simulation": {
    "type": "transient",
    "stop_time": 10e-3,
    "timestep": 1e-6
  }
}
'''

# Simulate directly from JSON string
result2 = pulsim.simulate_json(netlist_json)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(result2.time * 1e3, result2.voltages["in"], 'b-', label='V_in')
plt.plot(result2.time * 1e3, result2.voltages["out"], 'r-', label='V_out')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('RC Filter with Pulse Input')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Summary

In this notebook, you learned how to:
- Create circuits programmatically using `pulsim.Circuit`
- Configure simulation options
- Run transient simulations
- Access and plot results
- Work with JSON netlists

Next steps:
- [Buck Converter Tutorial](02_buck_converter.ipynb)
- [Thermal Modeling](03_thermal_modeling.ipynb)
- [Parameter Sweeps](04_parameter_sweeps.ipynb)